In [ ]:
# API KEY를 환경 변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()

# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

In [ ]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"  # template 정의

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

In [ ]:
from langchain_openai import ChatOpenAI

# prompt를 PromptTemplate 객체를 사용하여 생성
prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()
chain = prompt | model

input 변수에 딕셔너리 형태로 {topic}과 그에 대한 입력값을 할당하고 invoke() 함수로 답변 출력

In [ ]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'로 설정
input = {"topic": "인공지능 모델의 학습 원리"}

# invoke 메서드를 사용하여 input을 전달해서 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

invoke() 대신 스트리밍 출력

In [ ]:
from langchain_teddynote.messages import stream_response

answer = chain.stream(input)
stream_response(answer)

프롬프트 템플릿 변수는 여러개 사용 가능

In [ ]:
from langchain_openai import ChatOpenAI

# prompt를 PromptTemplate 객체를 사용하여 생성
prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 {how} 설명해주세요")
model = ChatOpenAI()
chain = prompt | model

input 객체에 {how} 변수에 대한 입력값을 dictionary 형태로 지정하고 invoke() 실행

In [ ]:
input = {"topic": "인공지능 모델의 학습 원리", "how": "5살짜리도 이해할 수 있도록"}
chain.invoke(input)

In [ ]:
from langchain_openai import ChatOpenAI

# prompt를 PromptTemplate 객체를 사용하여 생성
prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해주세요")

model = ChatOpenAI()

input = {"topic": "인공지능 모델의 학습 원리"}

LangChain에서 LLM의 응답을 문자열로 변환하는 파서인 `StringOutputParser`를 import 하고 해당 객체 생성

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

체인에 출력 파서 추가. 이제 LLM이 생성한 답변은 출력 파서가 해석(파싱)

In [ ]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인 구성
chain = prompt | model | output_parser

invoke() 함수로 답변 받으면 문자열로 된 답변 얻음

In [ ]:
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

In [39]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

프롬프트를 생성하고 모델과 출력 파서 초기화

In [40]:
# 프롬프트 템플릿을 이용하여 프롬프트 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서 초기화
output_parser = StrOutputParser()

chain = prompt | model | output_parser

완성된 체인을 invoke() 함수로 출력. 프롬프트 템플릿을 잘 활용하면 질문과 답변에서 바뀌는 부분만 변수로 지정하고 정해진 형식에 따라 생성 가능

In [41]:
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  Customer: Excuse me, could I see the menu, please?
  Waiter: Of course. Here's the menu. Would you like to start with something to drink?
  Customer: Yes, I’d like a glass of sparkling water, please.
  Waiter: Sparkling water, great choice. Are you ready to order, or would you like a few minutes?
  Customer: I’ll need a few more minutes, thank you.

- 한글 해석:
  고객: 실례합니다, 메뉴판 좀 볼 수 있을까요?
  웨이터: 물론입니다. 여기 메뉴판입니다. 음료로 무엇을 시작하시겠습니까?
  고객: 네, 탄산수 한 잔 주세요.
  웨이터: 탄산수, 좋은 선택이네요. 바로 주문하실래요, 아니면 조금 더 시간이 필요하신가요?
  고객: 조금 더 시간이 필요할 것 같아요, 감사합니다.


이번에는 {question} 변수를 변경하고 스트리밍 출력

In [42]:
from langchain_teddynote.messages import stream_response

answer = chain.stream({"question": "미국에서 피자 주문"})
stream_response(answer)

- 영어 회화:
  A: Hello, I'd like to order a pizza for delivery, please.
  B: Sure! What size would you like?
  A: I'd like a large pizza.
  B: What toppings would you like on that?
  A: Could I have pepperoni, mushrooms, and extra cheese?
  B: Absolutely, anything else?
  A: Yes, could I also have a side of garlic bread?
  B: Sure, one side of garlic bread. Anything to drink?
  A: A bottle of Coke, please.
  B: Alright, that's a large pizza with pepperoni, mushrooms, extra cheese, a side of garlic bread, and a bottle of Coke. Your total comes to $24.50. May I have your address?
  A: It’s 123 Elm Street.
  B: Great, your order will be there in about 30-40 minutes.
  A: Thank you!

- 한글 해석:
  A: 안녕하세요, 배달로 피자 하나 주문하고 싶습니다.
  B: 네! 피자 크기는 어떻게 하시겠어요?
  A: 라지 사이즈로 부탁드립니다.
  B: 어떤 토핑을 올려 드릴까요?
  A: 페퍼로니, 버섯 그리고 치즈 추가해 주세요.
  B: 알겠습니다, 추가로 더 필요한 것이 있나요?
  A: 네, 갈릭 브레드도 하나 주문할게요.
  B: 알겠습니다, 갈릭 브레드 하나요. 음료는 어떤 것으로 드릴까요?
  A: 콜라 한 병 부탁드립니다.
  B: 알겠습니다. 라지 사이즈 피자에 페퍼로니, 버섯, 치즈 추가, 갈릭 브레드, 콜라 한 병으로 